You can run this notebook directly on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DaniAffCH/Vessel-Geometric-Transformers/blob/main/main.ipynb)

In [ ]:
import sys
COLAB_RUNTIME = 'google.colab' in sys.modules
if COLAB_RUNTIME:
    !git clone https://github.com/DaniAffCH/Vessel-Geometric-Transformers.git
    !mv Vessel-Geometric-Transformers/* . 
    !pip install -q -r requirements.txt
else:
    !pip install -q -r requirements.txt
    !pre-commit autoupdate
    !pre-commit install

In [1]:
from src.utils import load_config
import os
from config import DatasetConfig

config_path = os.path.join("config","config.yaml")

config = load_config(config_path)
dataset_config: DatasetConfig = config.dataset

In [2]:
from src.data.dataset import get_datasets

data_dict = get_datasets(dataset_config)

In [7]:
# Testing 
from src.lib.geometricAlgebra import PointGeometricAlgebra, TranslationGeometricAlgebra, ScalarGeometricAlgebra
print(PointGeometricAlgebra.fromElement(data_dict["train"][0].pos))
print()

print(TranslationGeometricAlgebra.fromElement(data_dict["train"][1].wss)[:, :8])
print()

print(ScalarGeometricAlgebra.fromElement(data_dict["train"][0].pressure))
print()


tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0041,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0027,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0026,  1.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0083,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  2.2243,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0081,  1.0000,  0.0000]])

tensor([[ 1.0000,  0.0000,  0.0000,  ..., -1.3098, -3.7878, -0.7568],
        [ 1.0000,  0.0000,  0.0000,  ..., -1.3070, -3.7272, -0.7964],
        [ 1.0000,  0.0000,  0.0000,  ..., -1.2395, -3.6962, -0.8062],
        ...,
        [ 1.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

tensor([[133372.5312,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133347.1875,      